In [ ]:
from numpy import asarray
from numpy.random import randn
from numpy.random import randint
from keras.models import load_model
import matplotlib.pyplot as plt
from matplotlib import pyplot
from google.colab import drive
import numpy as np
import sklearn.linear_model as linear_model
drive.mount('/content/drive')

In [ ]:
# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
	# generate points in the latent space (Gaussian Distribution)
	x_input = randn(latent_dim * n_samples)
	# reshape into a batch of inputs for the generator
	z_input = x_input.reshape(n_samples, latent_dim)
	return z_input

# load model
model_male = load_model('male_generator_model.h5')
model_female = load_model('female_generator_model.h5')
# generate latent points
latent_points = generate_latent_points(100, 30000)
# generate images
X_male  = model_male.predict(latent_points)
X_female  = model_female.predict(latent_points)

In [ ]:
latents=np.array(latent_points)
print(latents.shape)

In [ ]:
# These are the attributes we are going to classify and ultimately be able to customize in our generated images
attributes1=['Black_Hair','No_Beard','Gray_Hair','Blond_Hair', 'Mustache']
attributes=['Male','Chubby','Young','Receding_Hairline','Pale_Skin','Straight_Hair','High_Cheekbones','Narrow_Eyes','Oval_Face','Wavy_Hair','Smiling']

In [ ]:
final_preds_male=np.zeros(30000)
for x in range(len(attributes)):
  str='/content/drive/My Drive/AML-Proj/'+attributes[x]+'_Model.h5'  # Generate model names so that we can load them from google drive
  model=load_model(str)
  prediction=model.predict(X_male)
  att_val=[]
  for i in range(30000):
    if prediction[i][0]>prediction[i][1]:  # Our CNN gives 2 values as output. If 1st is bigger attribute = 1, otherwise attribute = -1. However, instead of using +1 and -1 values we use the softmax values generated by the CNN since they as continuous and better suited for the regression model.
      att_val.append(prediction[i][0])
    else:
      att_val.append(-prediction[i][1])
  arr=np.asarray(att_val)
  final_preds_male=np.column_stack((final_preds_male,arr)) # Stacking all attribute columns to make one final attribute predictions array

for x in range(len(attributes1)):
  str='/content/drive/My Drive/AML-Proj-Attributes/'+attributes1[x]+'_Model.h5'  # Generate model names so that we can load them from google drive
  model=load_model(str)
  prediction=model.predict(X_male)
  att_val=[]
  for i in range(30000):
      att_val.append(prediction[i])
  arr=np.asarray(att_val)
  final_preds_male=np.column_stack((final_preds_male,arr)) # Stacking all attribute columns to make one final attribute predictions array

In [ ]:
final_preds_female=np.zeros(30000)
for x in range(len(attributes)):
  str='/content/drive/My Drive/AML-Proj/'+attributes[x]+'_Model.h5'  # Generate model names so that we can load them from google drive
  model=load_model(str)
  prediction=model.predict(X_female)
  att_val=[]
  for i in range(30000):
    if prediction[i][0]>prediction[i][1]:  # Our CNN gives 2 values as output. If 1st is bigger attribute = 1, otherwise attribute = -1. However, instead of using +1 and -1 values we use the softmax values generated by the CNN since they as continuous and better suited for the regression model.
      att_val.append(prediction[i][0])
    else:
      att_val.append(-prediction[i][1])
  arr=np.asarray(att_val)
  final_preds_female=np.column_stack((final_preds_female,arr)) # Stacking all attribute columns to make one final attribute predictions array

for x in range(len(attributes1)):
  str='/content/drive/My Drive/AML-Proj-Attributes/'+attributes1[x]+'_Model.h5'  # Generate model names so that we can load them from google drive
  model=load_model(str)
  prediction=model.predict(X_female)
  att_val=[]
  for i in range(30000):
      att_val.append(prediction[i])
  arr=np.asarray(att_val)
  final_preds_female=np.column_stack((final_preds_female,arr)) # Stacking all attribute columns to make one final attribute predictions array

In [ ]:
z=latents
y_male=final_preds_male
y_female=final_preds_female
y_male=np.delete(y_male,0,1)  # Removing the zero column
y_female=np.delete(y_female,0,1)  # Removing the zero column
print(z.shape)

In [ ]:
attributes=attributes+attributes1
attributes

# Regression Model

In [ ]:
def find_feature_axis(z, y):
  model_reg = linear_model.LinearRegression()
  print("Z",z.shape)
  print("y",y.shape)
  model_reg.fit(z, y)
  return model_reg.coef_.transpose()

def normalize_feature_axis(feature_slope):
    feature_direction = feature_slope / np.linalg.norm(feature_slope, ord=2, axis=0, keepdims=True)
    return feature_direction

In [ ]:
# regression: use latent space z to predict features y
feature_slope = find_feature_axis(z, y_female)

# normalize the feature vectors
feature_direction = normalize_feature_axis(feature_slope)

In [ ]:
import skimage.transform
latent_points=generate_latent_points(100,10)
images = model_female.predict(latent_points)

w=10
h=10
fig=plt.figure(figsize=(15, 15))
columns = 5
rows = 2
k=0
for i in range(1, columns*rows +1):
  fig.add_subplot(rows, columns, i)
  plt.imshow(images[k])
  k=k+1
plt.show()

In [ ]:
print(attributes)

# Using the regression model to change the facial features

In [ ]:
k=11
im_num=4

indexB=[]
feature_correlation = np.corrcoef(feature_direction.transpose())
for i in range(len(attributes)):
  if(i!=k and feature_correlation[k][i]>0.4):
    indexB.append(i)
print(indexB)

latent_points1 = latent_points + feature_direction[:, k]*1
latent_points2 = latent_points + feature_direction[:, k]*2
latent_points3 = latent_points + feature_direction[:, k]*3

image_new1  = model_female.predict(latent_points1)
image_new2  = model_female.predict(latent_points2)
image_new3  = model_female.predict(latent_points3)

prints=[images[im_num],image_new1[im_num],image_new2[im_num],image_new3[im_num]]

w=10
h=10
fig=plt.figure(figsize=(15, 15))
columns = 4
rows = 1
k=0
for i in range(1, columns*rows +1):
  img = np.random.randint(10, size=(h,w))
  fig.add_subplot(rows, columns, i)
  plt.imshow(prints[k])
  k=k+1
plt.show()

In [ ]:
def plot_feature_correlation(feature_direction, feature_name=None):
    import matplotlib.pyplot as plt

    len_z, len_y = feature_direction.shape
    if feature_name is None:
        feature_name = attributes

    feature_correlation = np.corrcoef(feature_direction.transpose())

    c_lim_abs = np.max(np.abs(feature_correlation))

    plt.pcolormesh(np.arange(len_y+1), np.arange(len_y+1), feature_correlation,
                   cmap='coolwarm', vmin=-c_lim_abs, vmax=+c_lim_abs)
    plt.gca().invert_yaxis()
    plt.colorbar()
    # plt.axis('square')
    plt.xticks(np.arange(len_y) + 0.5, feature_name, fontsize='x-small', rotation='vertical')
    plt.yticks(np.arange(len_y) + 0.5, feature_name, fontsize='x-small')
    plt.show()